# Stress Level Predictor

### This notebook shows the core functionalities of this project by creating and training different models. Model created can be trained and saved as pre-trained model for future usage. 

In [1]:
#Loading Required Libraries

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import cv2, os 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist
from imutils.video import FileVideoStream
from imutils.video import VideoStream
from imutils import face_utils
import argparse
import imutils
import time
import dlib
import pickle
import csv
import os.path

Using TensorFlow backend.


## Emotion Detection Part

In [13]:
classes = 7 #Total number of Expressions
epochs = 15
batch_size= 128

In [14]:
with open("fer2013.csv") as f:
    content = f.readlines()

lines = np.array(content)

instances = lines.size

print("Instances: ",instances)
print("Instance length: ",len(lines[1].split(",")[1].split(" ")))

Instances:  35888
Instance length:  2304


In [15]:
x_train, y_train, x_test, y_test = [], [], [], [] #Creating null arrays for testing and training data

In [16]:
for i in range(1,instances):
    try:
        emotion, pixel, type_class = lines[i].split(",")
          
        val = pixel.split(" ")
            
        pixels_float = np.array(val, 'float32')
        
        emotion = keras.utils.to_categorical(emotion, classes)
    
        if 'Training' in type_class:
            
            y_train.append(emotion)
            x_train.append(pixels_float)
            
        elif 'PublicTest' in type_class:
            
            y_test.append(emotion)
            x_test.append(pixels_float)
            
    except:
        print("",end="")

In [17]:
#Putting values in numpy arrays

x_train = np.array(x_train, 'float32')
y_train = np.array(y_train, 'float32')
x_test = np.array(x_test, 'float32')
y_test = np.array(y_test, 'float32')

#Normalizing the values
x_train /= 255
x_test /= 255

x_train = x_train.reshape(x_train.shape[0], 48, 48, 1)
x_train = x_train.astype('float32')

x_test = x_test.reshape(x_test.shape[0], 48, 48, 1)
x_test = x_test.astype('float32')

print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

28709 train samples
3589 test samples


In [18]:
model = Sequential()

#1st convolution layer
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(5,5), strides=(2, 2)))

#2nd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

#3rd convolution layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(AveragePooling2D(pool_size=(3,3), strides=(2, 2)))

model.add(Flatten())

#fully connected neural networks
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(classes, activation='softmax'))

In [19]:
#batch process
gen = ImageDataGenerator()
train_generator = gen.flow(x_train, y_train, batch_size=batch_size)

In [20]:
model.compile(loss='categorical_crossentropy'
    , optimizer=keras.optimizers.Adam()
    , metrics=['accuracy']
)

In [21]:
fit = True

if fit == True:
    #model.fit_generator(x_train, y_train, epochs=epochs) #train for all trainset
    model.fit_generator(train_generator, steps_per_epoch=batch_size, epochs=epochs) #train for randomly selected one
else:
    model.load_weights('facial_expression_model_weights.h5') #load weights

Epoch 1/15
128/128 [==============================] - 16s 128ms/step - loss: 1.8230 - acc: 0.2445
Epoch 2/15
128/128 [==============================] - 15s 117ms/step - loss: 1.8010 - acc: 0.2518
Epoch 3/15
128/128 [==============================] - 15s 117ms/step - loss: 1.7380 - acc: 0.2808
Epoch 4/15
128/128 [==============================] - 15s 118ms/step - loss: 1.6549 - acc: 0.3374
Epoch 5/15
128/128 [==============================] - 15s 118ms/step - loss: 1.5974 - acc: 0.3646
Epoch 6/15
128/128 [==============================] - 15s 117ms/step - loss: 1.5373 - acc: 0.3957
Epoch 7/15
128/128 [==============================] - 15s 118ms/step - loss: 1.5204 - acc: 0.4060
Epoch 8/15
128/128 [==============================] - 15s 117ms/step - loss: 1.4465 - acc: 0.4380
Epoch 9/15
128/128 [==============================] - 15s 117ms/step - loss: 1.4246 - acc: 0.4449
Epoch 10/15
128/128 [==============================] - 15s 118ms/step - loss: 1.3786 - acc: 0.4655
Epoch 11/15
128/128

In [ ]:
#overall evaluation
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', 100*score[1])

 384/3589 [==>...........................] - ETA: 2s

## Face Recognition part


Following are the names of subjects on which the face recognition model has been trained on. More names can be added
here as model learns to predict on more faces.


In [ ]:
#Names of subjects for face recognition part
subjects = ["", "Anubhav Kumar","Hemant Toshniwal","Arvind Kumar","Gitesh Bhasin"]

In [ ]:
#function to detect face using OpenCV
def detect_face(img):
    #converting the test image to gray image as opencv face detector expects gray images
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #loading OpenCV face detector, I am using LBP which is fast
    face_cascade = cv2.CascadeClassifier('lbpcascade_frontalface.xml')

    #Detecting multiscale (some images may be closer to camera than others) images
    #result is a list of faces
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=5);
    
    #if no faces are detected then return original img
    if (len(faces) == 0):
        return None, None
    
    #under the assumption that there will be only one face,
    #extracting the face area
    (x, y, w, h) = faces[0]
    
    #returning only the face part of the image
    return gray[y:y+w, x:x+h], faces[0]

In [ ]:
#this function will read all persons' training images, detect face from each image
#and will return two lists of exactly same size, one list 
# of faces and another list of labels for each face
def prepare_training_data(data_folder_path):
    
    #------STEP-1--------
    #getting the directories (one directory for each subject) in data folder
    dirs = os.listdir(data_folder_path)
    
    #list to hold all subject faces
    faces = []
    #list to hold labels for all subjects
    labels = []
    
    #Going through each directory and reading images within it
    for dir_name in dirs:
        
        #our subject directories start with letter 's' so
        #ignoring any non-relevant directories if any
        if not dir_name.startswith("s"):
            continue;
            
        #------STEP-2--------
        #extracting label number of subject from dir_name
        #format of dir name = slabel
        #so removing letter 's' from dir_name will give us label
        label = int(dir_name.replace("s", ""))
        
        #building path of directory containing images for current subject
        subject_dir_path = data_folder_path + "/" + dir_name
        
        #getting the images names that are inside the given subject directory
        subject_images_names = os.listdir(subject_dir_path)
        
        #------STEP-3--------
        #going through each image name, read image, 
        #detecting and adding face to list of faces
        for image_name in subject_images_names:
            
            #ignore system files like .DS_Store
            if image_name.startswith("."):
                continue;
            
            #building image path
            image_path = subject_dir_path + "/" + image_name

            #reading image
            image = cv2.imread(image_path)
            
            #displaying an image window to show the image 
            cv2.imshow("Training on image...", image)
            cv2.waitKey(100)
            
            #detecting face
            face, rect = detect_face(image)
            
            #------STEP-4--------
            if face is not None:
                #adding face to list of faces
                faces.append(face)
                #adding label for this face
                labels.append(label)
            
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    cv2.destroyAllWindows()
    
    return faces, labels

In [ ]:
#preparing our training data
#data will be in two lists of same size
#one list will contain all the faces
#and other list will contain respective labels for each face
print("Preparing data...")
faces, labels = prepare_training_data("training-data")
print("Data prepared")

#printing total faces and labels
print("Total faces: ", len(faces))
print("Total labels: ", len(labels))

Here we are using **Local binary pattern** histogram from opencv module. It is the one out of three libraries in opencv
package that helps in face recognition. LBPH is a better pick over others because it is not affected by the illumination 
of images and tries to find local features in images rather than looking at all the images at once. Right now the only limitation of using this model is that we are unable to set a minimum threshold value. Due to this every subject is identified as one of the trained subject. 

In [ ]:
#creating our LBPH face recognizer 
face_recognizer = cv2.face.LBPHFaceRecognizer_create()

In [ ]:
#training our face recognizer of our training faces
face_recognizer.train(faces, np.array(labels))

In [ ]:
#function to draw rectangle on image 
def draw_rectangle(img, rect):
    (x, y, w, h) = rect
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    
#function to draw text on give image
def draw_text(img, text, x, y):
    cv2.putText(img, text, (x, y), cv2.FONT_HERSHEY_PLAIN, 1.5, (0, 255, 0), 2)


In [ ]:
#this function recognizes the person in image passed
#and draws a rectangle around detected face with name of the 
#subject
def predict(test_img):
    #make a copy of the image as we don't want to chang original image
    img = test_img.copy()
    #detect face from the image
    face, rect = detect_face(img)

    #predict the image using our face recognizer 
    label= face_recognizer.predict(face)
    #get name of respective label returned by face recognizer
    label_text = subjects[label]
    #label_text = label
    #draw a rectangle around face detected
    draw_rectangle(img, rect)
    #draw name of predicted person
    draw_text(img, label_text, rect[0], rect[1]-5)
    
    return img


**Uses of each function**

- **eye_aspect_ratio** : This function helps in calculating the euclidean distance between the vertical and horizontal landmarks of of eye hence giving us eye aspect ratio. 


- **aperture** : This function helps in calculating the area to a given eye (Left eye in our case).It does so using the co-ordinates of landmarks of left eye


- **write_csv** : This function writes name of the subject, emotion, percentage of each emotion, blink rate of subject and area of the eye.


In [ ]:
#Function Definitions

#Calculates distance between vertical and horizontal eye landmarks
def eye_aspect_ratio(eye):
    # compute the euclidean distances between the two sets of
    # vertical eye landmarks coordinates
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # compute the euclidean distance between the horizontal
    # eye landmark coordinates
    C = dist.euclidean(eye[0], eye[3])

    # compute the eye aspect ratio
    ear = (A + B) / (2.0 * C)

    # return ear
    return ear

#Calculates eye aperture (Area of eye)
def aperture(xx,yy):
    f=[]
    g=0
    a=0
    for i in range(0,6):
        if i<5:
            g += xx[i]*yy[i+1]
            a += yy[i]*xx[i+1]
        else:
            g +=xx[i]*yy[i-5]
            a +=xx[i-5]*yy[i]
    d=abs(g-a)/2
    f.append(d)
    
    return f

#For writing data to CSV
def write_csv(data):
    #if file exists
    if os.path.exists('result.csv'):
        with open('result.csv', 'a') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(data)
    #if file does not exists
    else:
        with open('result.csv', 'a') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(["name", "emotion", "percentage","blink","area"])
            writer.writerow(data)

In [ ]:
#Variables Definition 

# Minimum EAR Threshold 
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 3

# Initializing the frame counters and the total number of blinks
COUNTER = 0
TOTAL = 0


Using front face detector function from dlib module to detect front faces and loading pre-trained 68 face landmark 
predictor.


In [ ]:
#Loading facial landmark predictor 

# initializing dlib's face detector (HOG-based) and then create the facial landmark predictor
detector = dlib.get_frontal_face_detector()
predictor2 = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


Finding out the 6 co-ordinates of each eye in order to calculate area as well as blink rate.


In [ ]:
# Initializing the indexes for the left and right eye, respectively

(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]


Here we use haarcascade frontalface XML provided by opencv that helps in detecting the faces in the frames. VideoCapture 
function of opencv can take the input in real time using webcam or it can take the feed from a saved video.


In [ ]:
#Loading haarcascade XML to detect front face

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#Taking feed from video
cap = cv2.VideoCapture('happy.mp4')
#Taking feed from camera
#cap = cv2.VideoCapture(0)


Here all the models are implemented in real time. First the **emotion recognition model** takes the feed from video/camera and predicts the emotion of the subjects in frame. The detectMultiScale function from face cascade helps in detecting multiple faces in one frame. The frame is then resized into **48x48** pixels and then converted into the numpy array which is fed to the model in order to predict the emotion.

After this the **face recognition model** runs in order to predict the face found in the frame. The next model **plots 68 landmark** on the face of subjects and then the eye blink rate and eye area is calculated which is later used as a factor to calculate stress level.

After all the model finishes their first run all the **outputs are saved to a CSV file**. 


In [ ]:
#Real-Time Open-CV integration of all models

emotions = ('angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral')
summedaperture = []
t1=time.time()
while(True):
    
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    
    ##########################################
    frame2 = imutils.resize(frame, width=800,height=800)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    rects = detector(gray2, 0)
    ##########################################

    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2) #drawing rectangle to main image

        detected_face = frame[int(y):int(y+h), int(x):int(x+w)] #cropping detected face
        detected_face = cv2.cvtColor(detected_face, cv2.COLOR_BGR2GRAY) #transforming to gray scale
        detected_face = cv2.resize(detected_face, (48, 48)) #resizing to 48x48

        img_pixels = image.img_to_array(detected_face)
        img_pixels = np.expand_dims(img_pixels, axis = 0)

        img_pixels /= 255 #pixels are in scale of [0, 255]. normalizing all pixels in scale of [0, 1]

        predictions = model.predict(img_pixels) #storing probabilities of 7 expressions
        
        #Face recognition
        fpredictions = face_recognizer.predict(detected_face)
        label=fpredictions[0]
        label_text = subjects[label]
        
        

        #finding max indexed array 0: angry, 1:disgust, 2:fear, 3:happy, 4:sad, 5:surprise, 6:neutral
        max_index = np.argmax(predictions[0])

        emotion = emotions[max_index]
        #writing emotion text above rectangle
        cv2.putText(frame, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2)
        cv2.putText(frame, label_text, (int(x), int(y)+150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        write_csv([label_text,emotion,predictions])
        
        
        cv2.imshow('Facial Recognizer', frame)
        
        for rect in rects:
        # determining the facial landmarks for the face region, then
        # converting the facial landmark coordinates to a NumPy
        # array
            shape = predictor2(gray2, rect)
            shape = face_utils.shape_to_np(shape)
            left_eye=shape[42:48,]

            x=[]
            y=[]
            for k in range(0,6):
                a=left_eye[k][0]
                x.append(a)
                b=left_eye[k][1]
                y.append(b)
            summedaperture.extend(aperture(x,y))

            # extracting the left and right eye coordinates, then using the
            # coordinates to compute the eye aspect ratio for both eyes
            leftEye = shape[lStart:lEnd]
            rightEye = shape[rStart:rEnd]
            leftEAR = eye_aspect_ratio(leftEye)
            rightEAR = eye_aspect_ratio(rightEye)

            # averaging the eye aspect ratio together for both eyes
            ear = (leftEAR + rightEAR) / 2.0

            # computing the convex hull for the left and right eye, then
            # visualizing each of the eyes
            leftEyeHull = cv2.convexHull(leftEye)
            rightEyeHull = cv2.convexHull(rightEye)
            cv2.drawContours(frame2, [leftEyeHull], -1, (0, 255, 0), 1)
            cv2.drawContours(frame2, [rightEyeHull], -1, (0, 255, 0), 1)

            # checking to see if the eye aspect ratio is below the blink
            # threshold, and if so, incrementing the blink frame counter
            if ear < EYE_AR_THRESH:
                COUNTER += 1

            # otherwise, the eye aspect ratio is not below the blink
            # threshold
            else:
                # if the eyes were closed for a sufficient number of
                # then incrementing the total number of blinks
                if COUNTER >= EYE_AR_CONSEC_FRAMES:
                    TOTAL += 1

                # resetting the eye frame counter
                COUNTER = 0

            # drawing the total number of blinks on the frame along with
            # the computed eye aspect ratio for the frame
            cv2.putText(frame2, "Blinks: {}".format(TOTAL), (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame2, "EAR: {:.2f}".format(ear), (300, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
    # showing the frame
    cv2.imshow("Eye Counter", frame2)
    if len(summedaperture)!=0:
        average_aperture=sum(summedaperture)/len(summedaperture)
    
    t2=time.time()
    if cv2.waitKey(1) & 0xFF == ord('q') or (t2-t1)>70: #press q to quiit
         
        write_csv([" "," ","",TOTAL,average_aperture])
        break
        
        
#killing open cv things
cap.release()
cv2.destroyAllWindows()


The below piece of code reads the csv file which we saved earlier and then extracts the most dominating emotion, blink
rate and eye area. 


In [ ]:
#loading CSV File
file=pd.read_csv('result.csv',header=0)

#Extracting most common feature from CSV file
emotion= file['emotion']
emotion_count = emotion.value_counts()
max_emotion= emotion_count.index[0]

#Extracting Blink Rate from CSV file
blink_rate = file['blink']
br=blink_rate.dropna()
br=br.values[0]

#Extracting Eye Area from CSV file
eye_area = file['area']
ear_area= eye_area.dropna()
area=ear_area.values[0]


Finally we classify the stress level based on the output of above models. We classify all the emotions into three 
categories. 

- **Not Stressed : If the emotion recognition model predicts that the most dominating emotion is happy then, we straight away classify our subject as not stressed.**

- **Mild Stress : If the emotion recognition model predicts that the most dominating emotion is neutral or anger, sad, fear then, we look at below factors:**

1) If the identified emotion is neutral and either the eye area lies between 350 to 530 or blink rate per minute lies between 25 to 46 then we say that the subject is under mild stress.
2) If the identified emotion is anger,sad, or fear and the eye area does not lies between 350 to 530 and blink rate per minute does not lies between 25 to 46 then we say that the subject is under mild stress.

- **Moderate Stress : If the emotion recognition model predicts that the most dominating emotion is neutral then, we look at below factors:**

1) If the eye area lies between 350 to 530 and blink rate per minute lies between 25 to 46 then, we say that the subject is under high stress.
2) If either of above condition is true then we say that the subject is moderately stressed.

- **High Stress : If the emotion recognition model predicts that the most dominating emotion is anger, sad, fear then, we look at below factors:**

1) If the eye area lies between 350 to 530 and blink rate per minute lies between 25 to 46 then, we say that the subject is under high stress.
2) If either of above condition is true then we say that the subject is moderately stressed.


In [1]:
#Final Classification of stress based on various factors and two predictor models
def StressClass(max_emotion,br,ear):
    happy = 0
    neutral = 0
    other = 0

    #Ranging values
    if max_emotion == "happy":
        happy = 1
    elif max_emotion == "neutral":
        neutral = 1 
    else:
        other = 1

    # If blink rate is between 25 to 46
    if br > 25 and br < 46:
        br=1
    else:
        br=0

    # If eye aperture is between 350 to 530
    if area>350 and area<530:
        ear=1
    else:
        ear=0

    #Stress Prediction
    if happy == 1:
        stress = "Not Stressed (Happy)"
    elif neutral == 1:
        if ear == 0 and br == 0:
            stress ="Not Stressed (Happy)"
        elif ear == 1 and br == 1:
            stress = "Moderate stress"
        else :
            stress = "Mild stress"
    else :
        if ear == 0 and br == 0:
            stress ="Mild stress"
        elif ear == 1 and br == 1:
            stress = "High stress"
        else :
            stress = "Moderate stress"
    return stress

# Final Output

In [ ]:
StressClass(max_emotion,br,ear)